## Initial Imports

In [1]:
import multiprocessing
from scm.plams import JobRunner, config, from_smiles, Settings, AMSJob, init

# this line is not required in AMS2025+
init()

PLAMS working folder: /path/plams/examples/BasisSetBenchmark/plams_workdir


## Set Up Job Runner
Set up job runner, running as many jobs as possible in parallel.

In [2]:
config.default_jobrunner = JobRunner(parallel=True, maxjobs=multiprocessing.cpu_count())
config.job.runscript.nproc = 1

## Set Up Molecules
Create the molecules we want to use in our benchmark from SMILES.

In [3]:
# The molecules we want to use in our benchmark:
mol_smiles = {"Methane": "C", "Ethane": "C-C", "Ethylene": "C=C", "Acetylene": "C#C"}
molecules = {}
for name, smiles in mol_smiles.items():
    # Compute 10 conformers, optimize with UFF and pick the lowest in energy.
    molecules[name] = from_smiles(smiles, nconfs=10, forcefield="uff")[0]
    print(name, molecules[name])

Methane   Atoms: 
    1         C       0.000000       0.000000       0.000000
    2         H       0.538912       0.762358      -0.599295
    3         H       0.731244      -0.596616       0.583182
    4         H      -0.567129      -0.670302      -0.678108
    5         H      -0.703028       0.504560       0.694220
  Bonds: 
   (1)--1.0--(2)
   (1)--1.0--(3)
   (1)--1.0--(4)
   (1)--1.0--(5)

Ethane   Atoms: 
    1         C      -0.757196      -0.040522       0.044605
    2         C       0.757196       0.040522      -0.044605
    3         H      -1.205222       0.185290      -0.945970
    4         H      -1.130281       0.694397       0.788688
    5         H      -1.061719      -1.061491       0.357407
    6         H       1.205222      -0.185290       0.945971
    7         H       1.130281      -0.694396      -0.788689
    8         H       1.061719       1.061491      -0.357406
  Bonds: 
   (1)--1.0--(2)
   (1)--1.0--(3)
   (1)--1.0--(4)
   (1)--1.0--(5)
   (2)--1.0--(6

## Initialize Calculation Settings
Set up the settings which are common across jobs. The basis type is added later for each job.

In [4]:
common_settings = Settings()
common_settings.input.ams.Task = "SinglePoint"
common_settings.input.ams.System.Symmetrize = "Yes"
common_settings.input.adf.Basis.Core = "None"

In [5]:
basis = ["QZ4P", "TZ2P", "TZP", "DZP", "DZ", "SZ"]
reference_basis = "QZ4P"

## Run Calculations

In [6]:
results = {}
for bas in basis:
    for name, molecule in molecules.items():
        settings = common_settings.copy()
        settings.input.adf.Basis.Type = bas
        job = AMSJob(name=name + "_" + bas, molecule=molecule, settings=settings)
        results[(name, bas)] = job.run()

[10.02|15:01:11] JOB Methane_QZ4P STARTED
[10.02|15:01:11] JOB Ethane_QZ4P STARTED
[10.02|15:01:11] JOB Ethylene_QZ4P STARTED
[10.02|15:01:11] JOB Methane_QZ4P RUNNING
[10.02|15:01:11] JOB Acetylene_QZ4P STARTED
[10.02|15:01:11] JOB Methane_TZ2P STARTED
[10.02|15:01:11] JOB Ethane_TZ2P STARTED
[10.02|15:01:11] JOB Ethylene_TZ2P STARTED
[10.02|15:01:11] JOB Acetylene_TZ2P STARTED
[10.02|15:01:11] JOB Methane_TZP STARTED
[10.02|15:01:11] JOB Ethane_QZ4P RUNNING
[10.02|15:01:11] JOB Ethylene_QZ4P RUNNING
[10.02|15:01:11] JOB Ethane_TZP STARTED
[10.02|15:01:11] JOB Ethylene_TZP STARTED
[10.02|15:01:11] JOB Acetylene_QZ4P RUNNING
[10.02|15:01:11] JOB Acetylene_TZP STARTED
[10.02|15:01:11] JOB Ethane_TZ2P RUNNING
[10.02|15:01:11] JOB Methane_DZP STARTED
[10.02|15:01:11] JOB Methane_TZ2P RUNNING
[10.02|15:01:11] JOB Ethane_DZP STARTED
[10.02|15:01:11] JOB Ethylene_DZP STARTED
[10.02|15:01:11] JOB Acetylene_DZP STARTED
[10.02|15:01:11] JOB Acetylene_TZ2P RUNNING
[10.02|15:01:11] JOB Methane_DZ

## Results
Extract the energy from each calculation. Calculate the average absolute error in bond energy per atom for each basis set.

In [7]:
average_errors = {}
for bas in basis:
    if bas != reference_basis:
        errors = []
        for name, molecule in molecules.items():
            reference_energy = results[(name, reference_basis)].get_energy(unit="kcal/mol")
            energy = results[(name, bas)].get_energy(unit="kcal/mol")
            errors.append(abs(energy - reference_energy) / len(molecule))
            print("Energy for {} using {} basis set: {} [kcal/mol]".format(name, bas, energy))
        average_errors[bas] = sum(errors) / len(errors)

[10.02|15:01:11] JOB Acetylene_TZP RUNNING
[10.02|15:01:11] Waiting for job Methane_QZ4P to finish
[10.02|15:01:11] JOB Methane_DZ RUNNING
[10.02|15:01:11] JOB Ethane_DZP RUNNING
[10.02|15:01:11] JOB Ethylene_DZP RUNNING
[10.02|15:01:11] JOB Acetylene_DZP RUNNING
[10.02|15:01:11] JOB Methane_SZ RUNNING
[10.02|15:01:11] JOB Ethane_DZ RUNNING
[10.02|15:01:11] JOB Ethane_SZ RUNNING
[10.02|15:01:11] JOB Ethylene_DZ RUNNING
[10.02|15:01:11] JOB Acetylene_DZ RUNNING
[10.02|15:01:11] JOB Acetylene_SZ RUNNING
[10.02|15:01:11] JOB Ethylene_SZ RUNNING
[10.02|15:01:15] JOB Methane_QZ4P FINISHED
[10.02|15:01:15] JOB Methane_TZ2P FINISHED
[10.02|15:01:15] JOB Methane_TZ2P SUCCESSFUL
[10.02|15:01:15] JOB Methane_QZ4P SUCCESSFUL
Energy for Methane using TZ2P basis set: -572.110159165262 [kcal/mol]
[10.02|15:01:15] Waiting for job Ethane_QZ4P to finish
[10.02|15:01:15] JOB Methane_DZP FINISHED
[10.02|15:01:15] JOB Methane_TZP FINISHED
[10.02|15:01:15] JOB Methane_DZP SUCCESSFUL
[10.02|15:01:15] JOB Me

In [8]:
print("== Results ==")
print("Average absolute error in bond energy per atom")
for bas in basis:
    if bas != reference_basis:
        print("Error for basis set {:<4}: {:>10.3f} [kcal/mol]".format(bas, average_errors[bas]))

== Results ==
Average absolute error in bond energy per atom
Error for basis set TZ2P:      0.170 [kcal/mol]
Error for basis set TZP :      0.537 [kcal/mol]
Error for basis set DZP :      1.024 [kcal/mol]
Error for basis set DZ  :      3.339 [kcal/mol]
Error for basis set SZ  :     27.683 [kcal/mol]
